In [2]:
import os
os.chdir('../')

In [6]:
import numpy as np
import os
import skimage
from skimage import measure
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

from tqdm import tqdm
from collections import defaultdict
import itertools
import argparse
import pickle
import logging

In [35]:
directory_path = r'C:\Users\Wojtek\Documents\Doktorat\Astral\data\Cont_AA_2_4'

waves = np.load(os.path.join(directory_path, 'waves_morph.npy'))
volume_threshold = 45

In [36]:
def _indices_label(array, label, offset):
    indices = np.argwhere(array == label)
    indices = [np.concatenate([elem[:-1], [elem[-1] + offset]]).tolist() for elem in indices]
    return indices

In [37]:
def find_closest_slice(myList, myNumber): 
    return min(myList, key=lambda x: abs(x - myNumber))

In [50]:
def find_slice_points(waves, axis= -1):
    if axis == -1 or axis == 2:
        slices = [slic for slic in range(waves.shape[axis]) if not np.any(waves[:, :, slic])]
    elif axis == 1:
        slices = [slic for slic in range(waves.shape[axis]) if not np.any(waves[:, slic, :])]
    else:
        slices = [slic for slic in range(waves.shape[axis]) if not np.any(waves[slic, :, :])]
        
    length = waves.shape[axis]
    
    to_slice = [i*25 for i in range(int(length/25))]

    out = list(map(lambda x: find_closest_slice(slices, x), to_slice))
    out = [*out, length]
    out = sorted(list(set(out)))
    
    return out

In [57]:
slices = [slic for slic in range(waves.shape[1]) if not np.any(waves[:, slic, :])]

In [62]:
labels = np.load(r'C:\Users\Wojtek\Documents\Doktorat\MSI_methods\lables.npy')

In [59]:
a = np.load(r'F:\MassSpectrometryImaging\clustering\partition.npy')

In [61]:
np.unique(a)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106], dtype=int64)

In [48]:
a = find_slice_points(waves, axis=0)

608
[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575]
[]


ValueError: min() arg is an empty sequence

In [40]:
a

[193, 199, 625, 1200]

In [49]:
logging.debug('Running run method')

out = find_slice_points(waves, axis=-1)

total = []

for index in tqdm(range(len(out) - 1)):
    logging.debug(f'Starting {index} iteration')
    current = waves[:, :, out[index]:out[index + 1]]
    logging.debug(f'Labelling objects in a subspace...')
    logging.debug(current.dtype)
    labelled = measure.label(current, connectivity=3).astype('uint16')
    logging.debug(f'Finished labelling!')
    last_slice = index
    uniq, counts = np.unique(labelled, return_counts=True)
    labels = uniq[1:]
    counts = counts[1:]
    logging.debug(f'Got {len(labels)} labels')
    label_counts = list(zip(labels, counts))
    count_filtered = list(filter(lambda x: x[1] > volume_threshold, label_counts))
    if not count_filtered:
        continue
    labels, counts = zip(*count_filtered)
    object_cords = Parallel(n_jobs=3, verbose=10)(delayed(self._indices_label)
                                                  (labelled, label, out[index]) for label in labels)

    logging.debug(f'Finishing {index} iteration')

    total.extend(object_cords)

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

1200
[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 900, 925, 950, 975, 1000, 1025, 1050, 1075, 1100, 1125, 1150, 1175]
[90, 99, 108, 111, 113, 118, 120, 121, 123, 124, 126, 129, 130, 131, 132, 133, 134, 135, 139, 140, 155, 157, 163, 183, 187, 188, 202, 203, 204, 205, 215, 216, 217, 218, 219, 222, 273, 274, 275, 276, 277, 278, 306, 307, 340, 341, 349, 350, 354, 355, 357, 358, 359, 366, 367, 368, 369, 389, 390, 463, 535, 536, 537, 538]


NameError: name 'self' is not defined

In [35]:
waves = waves[..., :400]

In [140]:
neighbor_df = pd.read_csv(os.path.join(directory_path, 'neighbors.csv')).astype('int16')
abs_df = pd.read_hdf(os.path.join(directory_path, 'segmentation_absolute.h5')).astype('int16')

In [122]:
abs_df['id'].value_counts()

0      1127495
1       955412
2       436166
3       425105
4       401517
        ...   
532        205
534        203
533        203
535        201
536        201
Name: id, Length: 537, dtype: int64

In [61]:
def _get_shape_voxels_by_id(abs_csv, shape_id):
    return abs_csv.loc[abs_csv['id'] == shape_id]

In [62]:
shape1 = _get_shape_voxels_by_id(abs_df, 0)

In [63]:
shape1.shape

(1127495, 5)

In [142]:
threshold = 0.8

In [154]:
s1 = _get_z_projection(0, abs_df)

In [87]:
neighbor_df.dtypes

shape_id_1                int64
shape_id_2                int64
center_dist_xy            int64
center_dist_z             int64
center_of_mass_dist_xy    int64
center_of_mass_dist_z     int64
dtype: object